In [2]:
# # # local
project_directory = "../"


# # # # colab
# from google.colab import drive
# drive.mount('/content/drive')
# project_directory = "/content/drive/MyDrive/colab_working_directory/diversity-enforced-ensembles/"
# !pip install cached-property

In [3]:
from pathlib import Path
import pandas as pd
import numpy as np

# allow import of decompose locally
import sys
sys.path.append(project_directory + 'src/')

from decompose import SquaredLoss
import bvdlib

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.func import stack_module_state
from torch.func import functional_call
from torch import vmap
import copy

from bvdlib.trial import Trial
from numpy.random import choice
from numpy.random import seed
from numpy import array
import numpy as np
import pickle

In [4]:
class SimpleConvnet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 2, 20)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(2, 4, 35)
        self.fc1 = nn.Linear(4 * 18 * 18, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # print(F.relu(self.conv1(x)).size())
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        print(x.size())
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        print(x.size())
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return self.softmax(x)

In [5]:
x = torch.rand((20, 1, 160, 160))
y = torch.randint(0,10,(20,), dtype=torch.float)

In [6]:
def CE_Combiner(x, axis=-1):
  log_x = torch.log(x)
  geometric_mean = torch.exp(torch.mean(log_x, dim=0))
  return geometric_mean

  # geo_mean = torch.exp(log_sum / x.shape[axis])
  # return geo_mean / x.sum(dim=axis)


In [7]:
mod = SimpleConvnet()
pred = mod(x)

ens_pred = pred.repeat(11,1,1)
ens_pred.size()

torch.Size([20, 4, 18, 18])
torch.Size([20, 1296])


torch.Size([11, 20, 10])

In [8]:
pred

tensor([[0.0934, 0.1044, 0.1026, 0.1092, 0.0954, 0.0897, 0.1031, 0.0969, 0.0932,
         0.1121],
        [0.0933, 0.1047, 0.1024, 0.1094, 0.0953, 0.0896, 0.1031, 0.0967, 0.0929,
         0.1125],
        [0.0931, 0.1044, 0.1025, 0.1095, 0.0955, 0.0897, 0.1030, 0.0968, 0.0931,
         0.1125],
        [0.0931, 0.1045, 0.1025, 0.1096, 0.0954, 0.0897, 0.1030, 0.0968, 0.0932,
         0.1122],
        [0.0933, 0.1044, 0.1026, 0.1093, 0.0954, 0.0896, 0.1032, 0.0970, 0.0930,
         0.1121],
        [0.0933, 0.1045, 0.1024, 0.1095, 0.0952, 0.0897, 0.1029, 0.0969, 0.0932,
         0.1124],
        [0.0934, 0.1046, 0.1026, 0.1094, 0.0953, 0.0896, 0.1031, 0.0967, 0.0931,
         0.1122],
        [0.0932, 0.1044, 0.1025, 0.1097, 0.0953, 0.0896, 0.1032, 0.0967, 0.0931,
         0.1122],
        [0.0934, 0.1044, 0.1027, 0.1094, 0.0952, 0.0898, 0.1030, 0.0970, 0.0931,
         0.1120],
        [0.0930, 0.1043, 0.1028, 0.1098, 0.0955, 0.0895, 0.1031, 0.0969, 0.0930,
         0.1122],
        [0

In [9]:
ens_out = CE_Combiner(ens_pred)
ens_out

tensor([[0.0934, 0.1044, 0.1026, 0.1092, 0.0954, 0.0897, 0.1031, 0.0969, 0.0932,
         0.1121],
        [0.0933, 0.1047, 0.1024, 0.1094, 0.0953, 0.0896, 0.1031, 0.0967, 0.0929,
         0.1125],
        [0.0931, 0.1044, 0.1025, 0.1095, 0.0955, 0.0897, 0.1030, 0.0968, 0.0931,
         0.1125],
        [0.0931, 0.1045, 0.1025, 0.1096, 0.0954, 0.0897, 0.1030, 0.0968, 0.0932,
         0.1122],
        [0.0933, 0.1044, 0.1026, 0.1093, 0.0954, 0.0896, 0.1032, 0.0970, 0.0930,
         0.1121],
        [0.0933, 0.1045, 0.1024, 0.1095, 0.0952, 0.0897, 0.1029, 0.0969, 0.0932,
         0.1124],
        [0.0934, 0.1046, 0.1026, 0.1094, 0.0953, 0.0896, 0.1031, 0.0967, 0.0931,
         0.1122],
        [0.0932, 0.1044, 0.1025, 0.1097, 0.0953, 0.0896, 0.1032, 0.0967, 0.0931,
         0.1122],
        [0.0934, 0.1044, 0.1027, 0.1094, 0.0952, 0.0898, 0.1030, 0.0970, 0.0931,
         0.1120],
        [0.0930, 0.1043, 0.1028, 0.1098, 0.0955, 0.0895, 0.1031, 0.0969, 0.0930,
         0.1122],
        [0

In [10]:
print(pred[0], ens_out[0])

tensor([0.0934, 0.1044, 0.1026, 0.1092, 0.0954, 0.0897, 0.1031, 0.0969, 0.0932,
        0.1121], grad_fn=<SelectBackward0>) tensor([0.0934, 0.1044, 0.1026, 0.1092, 0.0954, 0.0897, 0.1031, 0.0969, 0.0932,
        0.1121], grad_fn=<SelectBackward0>)


In [11]:
ens_out

tensor([[0.0934, 0.1044, 0.1026, 0.1092, 0.0954, 0.0897, 0.1031, 0.0969, 0.0932,
         0.1121],
        [0.0933, 0.1047, 0.1024, 0.1094, 0.0953, 0.0896, 0.1031, 0.0967, 0.0929,
         0.1125],
        [0.0931, 0.1044, 0.1025, 0.1095, 0.0955, 0.0897, 0.1030, 0.0968, 0.0931,
         0.1125],
        [0.0931, 0.1045, 0.1025, 0.1096, 0.0954, 0.0897, 0.1030, 0.0968, 0.0932,
         0.1122],
        [0.0933, 0.1044, 0.1026, 0.1093, 0.0954, 0.0896, 0.1032, 0.0970, 0.0930,
         0.1121],
        [0.0933, 0.1045, 0.1024, 0.1095, 0.0952, 0.0897, 0.1029, 0.0969, 0.0932,
         0.1124],
        [0.0934, 0.1046, 0.1026, 0.1094, 0.0953, 0.0896, 0.1031, 0.0967, 0.0931,
         0.1122],
        [0.0932, 0.1044, 0.1025, 0.1097, 0.0953, 0.0896, 0.1032, 0.0967, 0.0931,
         0.1122],
        [0.0934, 0.1044, 0.1027, 0.1094, 0.0952, 0.0898, 0.1030, 0.0970, 0.0931,
         0.1120],
        [0.0930, 0.1043, 0.1028, 0.1098, 0.0955, 0.0895, 0.1031, 0.0969, 0.0930,
         0.1122],
        [0

In [12]:
torch.allclose(ens_out, pred)

True

In [27]:
x = torch.rand((900,11))
y = torch.randint(0,10,(900,), dtype=torch.float)
crit = torch.nn.MSELoss()
with torch.no_grad():
    result = np.array(losses)

result[1]

In [15]:
save_path = project_directory + "experiments/results/dfsgfsdsdfg.pkl"

In [16]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

x = housing['data']
y = housing['target']

print("x shape", x.shape)
print("y shape", y.shape)

x shape (20640, 8)
y shape (20640,)


In [17]:
n_trials = 100
trial_space = np.arange(1,21) # Test estimators from 1 to 20
data_percentage_training = int(0.8 * len(y))
num_training = int(0.8 * data_percentage_training)

In [18]:
def init_layer(layer, generator):
    torch.nn.init.xavier_uniform_(layer.weight, generator=generator)
    layer.bias.data.fill_(0.01)

class SimpleMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, generator):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        init_layer(self.fc1, generator)

        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        init_layer(self.fc2, generator)

        self.fc3 = nn.Linear(hidden_dim, output_dim)
        init_layer(self.fc3, generator)

    def forward(self, x):

        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)

        return x

In [19]:
class trial_dataset(torch.utils.data.Dataset):
  def __init__(self, x, y, device):
    self.x = torch.tensor(x).type(torch.float).to(device)
    self.y = torch.tensor(y).type(torch.float).to(device)

  def __len__(self):
    return len(self.x)

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

In [20]:
experiment_seed = 0
np.random.seed(experiment_seed)
rng = np.random.default_rng()
shuffled_indices = rng.permutation(len(y))

train_indices = shuffled_indices[:data_percentage_training]
test_indices = shuffled_indices[data_percentage_training:]
train_data = x[train_indices, :]
train_labels = y[train_indices]
test_data = x[test_indices, :]
test_labels = y[test_indices]

In [21]:
def forward_through_metamodel(params, buffers, x):
        return functional_call(meta_model, (params, buffers), (x,))

def torch_MSE_combiner(ensemble_output):
    return torch.mean(ensemble_output, axis=0)

combiner_rule = torch_MSE_combiner

def ens_forward(input, ensemble):

  params, buffers = stack_module_state(nn.ModuleList(ensemble))

  member_output = vmap(forward_through_metamodel)(params, buffers, input.repeat(len(ensemble), 1, 1))

  ensemble_output = combiner_rule(member_output)
  return ensemble_output, member_output

In [22]:
trial_space = np.arange(0,15) / 10
decomp_fn = SquaredLoss
seed = 0
criterion = torch.nn.MSELoss()
epoch_n=7

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


torch_generator = torch.manual_seed(0)

test_dset = trial_dataset(test_data, test_labels, device)


test_loader = torch.utils.data.DataLoader(test_dset, batch_size = 258, shuffle=False)

train_dset = trial_dataset(train_data, train_labels, device)
train_dloader = torch.utils.data.DataLoader(train_dset, batch_size = 128, shuffle=True, generator=torch_generator)
train_unshuffled_dloader = torch.utils.data.DataLoader(train_dset, batch_size = 258, shuffle=False)


In [23]:
len_of_batch = 20

trial_x, trial_y = (torch.rand((len_of_batch,13), dtype=torch.float), torch.randint(0,10,(len_of_batch,),dtype=torch.float))

trial_dset = trial_dataset(trial_x, trial_y, device)
trial_dloader = torch.utils.data.DataLoader(trial_dset, batch_size = 128, shuffle=True, generator=torch_generator)
trial_unshuffled_dloader = torch.utils.data.DataLoader(trial_dset, batch_size = 258, shuffle=False)

# init model1
ensemble = []
optims = []
losses = []

n_estimators = 11
for member_n in range(n_estimators):
    ensemble.append(SimpleMLP(len(trial_x[0]), 6, 1, torch_generator).to(device))
    optims.append(torch.optim.Adam(ensemble[member_n].parameters()))
    losses.append(None)

global meta_model 
meta_model = copy.deepcopy(ensemble[0]).to('meta')

lambda_ = 0.3

/tmp/ipykernel_67196/506644903.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.x = torch.tensor(x).type(torch.float).to(device)
/tmp/ipykernel_67196/506644903.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y = torch.tensor(y).type(torch.float).to(device)


In [32]:
def hook(grad):
    print("Batch gradient",grad)
    # print("2nd grad", torch.autograd.grad(grad))

In [43]:
class Calc_Squared_Error_Loss(torch.nn.Module):
    def __init__(self, lambda_, n_estimators):
        super().__init__()
        self.lambda_ = lambda_
        self.m = n_estimators


    def forward(self, member_output, ensemble_output, label):
        loss = (torch.square(member_output - label)) - (self.lambda_ * torch.square(member_output - ensemble_output))
        gradient = 2*(member_output - label) - (self.lambda_ * 2 * (1-(1/self.m)) * (member_output - ensemble_output))
        return torch.mean(loss), gradient / member_output.size()[0]

loss_fn = torch.jit.script(Calc_Squared_Error_Loss(lambda_, n_estimators))

In [46]:
class Calc_Squared_Error_LossCC(torch.nn.Module):
    def __init__(self, lambda_, n_estimators):
        super().__init__()
        self.lambda_ = lambda_
        self.m = n_estimators


    def forward(self, member_output, ensemble_output, label):
        loss = ((0.5 * torch.square(member_output - label))) - (self.lambda_ * torch.square(member_output - ensemble_output))
        return torch.mean(loss)

loss_fn_hess = torch.jit.script(Calc_Squared_Error_LossCC(lambda_, n_estimators))

In [29]:
trial_x.shape

torch.Size([20, 13])

In [49]:

bx, by = trial_x.to(device), trial_y.to(device)

with torch.no_grad():
    ensemble_output, member_output = ens_forward(bx, ensemble)
ensemble_output_gradenable, member_output__gradenable = ens_forward(bx, ensemble)

member_output = member_output.detach()

for i, member in enumerate(ensemble):

    optims[i].zero_grad()
    member_pred = member(bx) #predict
    member_pred.register_hook(hook)
    member_grad_output = torch.cat((member_output[:i], member_pred.unsqueeze(dim=0), member_output[i+1:]))
    ens_grad_output = combiner_rule(member_grad_output)

    # member_loss = (criterion(member_pred, by.unsqueeze(dim=-1)) + ((lambda_) * criterion(ens_grad_output, member_pred)))
    member_loss = (criterion(member_pred, by.unsqueeze(dim=-1)) - ((lambda_) * criterion(member_pred, ens_grad_output)))
    
    # print(ens_grad_output.requires_grad)
    # member_loss = (criterion(member_pred, by.unsqueeze(dim=-1)) + ((lambda_) * criterion(member_pred, ensemble_output_gradenable)))
    # member_loss = criterion(member_pred, by.unsqueeze(dim=-1))
    loss, grad = loss_fn(member_pred, ensemble_output, by.unsqueeze(dim=-1))
    print("Hess",torch.autograd.functional.hessian(loss_fn_hess, (member_pred, ensemble_output, by.unsqueeze(dim=-1))))

    member_loss.backward()
    print("calcedgrad", grad)
    print("jitloss",loss)
    print("criterion Loss", member_loss)

    # member_loss.backward()
    non_touched_grad = 2*(member_pred - by.unsqueeze(dim=-1)) # correct
    ncl_grad = (2*(member_pred - by.unsqueeze(dim=-1)) - (lambda_*(2)*((n_estimators-1)/n_estimators)*(member_pred-ensemble_output))) /len_of_batch
    # member_pred.backward(ncl_grad)


    # print(non_touched_grad/2)
    print("Exp. ncl_grad", ncl_grad)
    print()


    # optims[i].step()

Hess ((tensor([[[[0.0200],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000]]],


        [[[0.0000],
          [0.0200],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000]]],


        [[[0.0000],
          [0.0000],
          [0.0200],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
          [0.0000],
     